# 1. Frame the problem
Given a picture of a random English cursive letter, determine which letter the picture depicts. (Ex. A short loop generally depicts an e, while a tall loop depicts an l.)

# 2. Get the Data 
- The data was initially provided in a google drive zip folder called "2023 Cursive".
- To initially load the data, I downloaded the zip file onto my local machine, and then utilized the Jupyter upload feature to upload the zip file to the remote server.
- After the zip file was fully uploaded, I unzipped the file using the unzip package, renaming the outputted folder to 2023_Cursive for simplicity. Inside the new folder there were 9 zip files, each labelled 1 to 9."
- I unzipped each folder individually using the unzip package. Each time I would unzip a folder, its name would change to what the student originally called it, so I would rename the newly created folder back to the number of its corresponding zip file. This was to keep the files/folders organized, also preventing files from mixing since some of the original folder names are identical. (Ex. Unzipping "1.zip" outputs a folder called "Cursive letters", so I renamed that folder to just "1".)
- Finally, after unzipping everything, I deleted all the zip files by running "rm *.zip -r" in the "2023_Cursive" folder, leaving only the newly unzipped folders.

# 3. Explore the Data

#### Below is a function that makes it easier to view images in the IPYNB:

In [2]:
def show(img_obj): #made this since pyplot only shows the correct color if u convert to rgb but everything in opencv is done in bgr so yea also this just helps with keeping code clean and non repeating
    plt.imshow(cv2.cvtColor(img_obj, cv2.COLOR_BGR2RGB))
    plt.show()

#### Below is a data explorer that just allows you to look at any image you want by simply typing a 2 character string: the number of the folder, and the letter. Alternatively, typing a single number or letter will show you the corresponding folder or all pictures of the corresponding letters.
- (Ex. "1a" will show you the picture of the letter "a" in folder 1. "a" will show you all pictures of the letter "a" across all 9 folders, and "1" will show you all pictures in folder 1.)

In [ ]:
import cv2
from matplotlib import pyplot as plt
from IPython.display import clear_output #so that output is cleared after every page to prevent clutter and huge outputs

#search thing to see any image without having to look for it in files
while True:
    uinput = input("Folder and Letter: ") #ex. "2a" to see folder 2, letter a. SPECIAL FEATURE: to see all images of a certain letter, do "s" + the letter you want to view, and it will show all letters.
    clear_output()
    print("Folder and Letter: " + uinput)
    if len(uinput) == 1:
        try:
            for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
                image = cv2.imread("2023_Cursive/" + str(int(uinput)) + "/" + letter + ".JPG")
                show(image)
        except:
            for i in range (1,10):
                image = cv2.imread("2023_Cursive/" + str(i) + "/" + uinput[0].upper() + ".JPG")
                show(image)
    else:
        image = cv2.imread("2023_Cursive/" + uinput[0] + "/" + uinput[1].upper() + ".JPG")
        show(image)

# 4. Prepare the Data
#### Some data prep was done during the unzipping process in step 2 to ensure the files and folders all remained organized and consistent enough to be exploreable (see step 2 for details).
#### The files themselves were still messy though, so some data prep had to be done between steps 2 and 3 in preparation for data exploration.
#### Since this data prep wasn't addressed between the 2 steps, all the issues fixed between steps 2 and 3 are listed below (in the same order they were resolved) and their solutions are provided:
- Folders 2 and 6 had several images with no file extensions.
    - To fix this, I created a short program (located in the same folder as this IPYNB) called rename.py. I moved this program into the folders with this issue, and upon running it, it would add ".HEIC" to the end of each file name in the folder (I also made sure to check beforehand that all files with this issue were HEIC images, so there was no need to distinguish between different file types, making the program very simple yet effective).
- Folder 8 had three versions of every image, two versions being unnamed.
    - To fix this, I just ran the command "rm IMG* -r" in the problematic directory which removed all files beginning with "IMG". This left only the files that were properly named.
- Only folders 3, 7, and 9 contained their images in a JPG format. The rest of the folders all contained HEIC images, which are unsupported by OpenCV and most other image processing libraries.
    - To fix this, I created a program (located in the same folder as this IPYNB) called convert.py. Upon being run, this program loops through every single image in the provided dataset. If an image is HEIC, it is converted into JPG. Also, for the sake of consistency, all file names are converted to uppercase so that all file names are the exact same for each set of cursive letters.
###### _At this point, I might as well just do step 4 before step 3 every time since data prep always happens there..._
#### Finally, we return to where we are now, step 4.
- Now arises a much more difficult issue to deal with: standardizing the images themselves.
- I plan to crop and scale the images to be the same resolution, aspect ratio, and have each letter in just about the same area. I'm also considering stretching the letters to keep the height and width of the letters consistent but idk yet
- My first step initially was to binarize the images, basically just rounding each pixel to be either black or white.
- After some online searches, it seems it would be better to just use grayscale for training, since it is more accurate.
#### Try 1:
- My first idea was to binarize the images, find lines in the binarized images, and if the lines made a box, I'd crop the image to the box.
- Then after cropping the image to the box, I'd crop the images to the edges of where black pixels are found.
- Finally, I'd get the cropped coordinates and crop the original image, plus resizing it to be a consistent resolution throughout all images
- The reason I couldn't get this to work is because it was simply taking too long to find out how to isolate the 4 most relevant lines. It could have worked, but I didn't have enough time to spend on debugging
- #### Try 2:
- My second (and more functional) idea was to blur the image slightly to make everything a bit smoother, 



#### Note for the deployment function: when the deployment function is made, the inputted data will have to be cleaned. I will have to run the inputted image through all the same procedures that the training data went through, so I will have to redo all this data cleaning in the deployment function.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import copy
import math
import os




def isolateLetter(gcroppedimage):
        
    cheight = gcroppedimage.shape[0]
    cwidth = gcroppedimage.shape[1]
    
    bcroppedimage = cv2.medianBlur(gcroppedimage,65) #warped image blurred the same way the og image is blurred

    cedges = cv2.Canny(bcroppedimage, 50, 70)
    # show(cedges)
    
    bincroppedimage = cv2.threshold(cedges, 140, 255, cv2.THRESH_BINARY)[1] #binarized
    # show(bincroppedimage)

    kernel = np.ones((5,5), np.uint8)
    dilcroppedimage = cv2.dilate(bincroppedimage, kernel, iterations=1)

    # invbinimage = cv2.bitwise_not(binimage) #invert img to work
    
    # zeros = np.transpose(np.nonzero(invbinimage)) #array of all nonzero value indexes (except since its inverted its actually the zero values)
    zeros = np.transpose(np.nonzero(dilcroppedimage))
    # show(dilcroppedimage)

    if np.min(zeros, axis=0)[0] < cheight/20 or np.max(zeros, axis=0)[0] > cheight-cheight/20 or np.min(zeros, axis=0, keepdims=True)[0][1] < cwidth/20 or np.max(zeros, axis=0, keepdims=True)[0][1] > cwidth-cwidth/20: #yo theres people who cropped their images but like left a really small amount of table so basically for those people check if the lettertop/bottom/l/r are 0 then if so just zoom in a bit
        dilcroppedimage = dilcroppedimage[int(cheight/20):int(cheight-cheight/20), int(cwidth/20):int(cwidth-cwidth/20)]
        gcroppedimage = gcroppedimage[int(cheight/20):int(cheight-cheight/20), int(cwidth/20):int(cwidth-cwidth/20)]
        zeros = np.transpose(np.nonzero(dilcroppedimage))
        print("cropped due to minor uncropped area")
        # show(dilcroppedimage)

    cheight = gcroppedimage.shape[0]
    cwidth = gcroppedimage.shape[1]

    
    letterTop = np.min(zeros, axis=0)[0]
    letterBottom = np.max(zeros, axis=0)[0]
    letterLeft = np.min(zeros, axis=0, keepdims=True)[0][1]
    letterRight = np.max(zeros, axis=0, keepdims=True)[0][1]

    

    letterHeight = letterBottom-letterTop
    letterWidth = letterRight-letterLeft
    # print("letter height: " + str(letterHeight))
    # print("letter width: " + str(letterWidth))

    
    if letterHeight > letterWidth:
        # print("height bigger")
        letterBottom += cheight/10
        letterTop -= cheight/10
        newLetterHeight = letterBottom-letterTop
        letterLeft -= (newLetterHeight-letterWidth)/2
        letterRight += (newLetterHeight-letterWidth)/2
    
    elif letterHeight < letterWidth:
        # print("width bigger")
        letterLeft -= cheight/10
        letterRight += cheight/10
        newLetterWidth = letterRight-letterLeft
        letterTop -= (newLetterWidth-letterHeight)/2
        letterBottom += (newLetterWidth-letterHeight)/2
        
    else:
        # print("height = width (damn)")
        letterBottom += cheight/10
        letterTop -= cheight/10
        letterLeft -= cheight/10
        letterRight += cheight/10

    letterBottom = max(0, letterBottom)
    letterTop = max(0, letterTop)
    letterLeft = max(0, letterLeft)
    letterRight = max(0, letterRight)
    
    # print("new letter height: " + str(letterBottom-letterTop))
    # print("new letter width: " + str(letterRight-letterLeft))

    # print("letterLeft: " + str(int(letterLeft)))
    # print("letterRight: " + str(int(letterRight)))
    # print("letterTop: " + str(int(letterTop)))
    # print("letterBottom: " + str(int(letterBottom)))

    
    letterimage = gcroppedimage[int(letterTop):int(letterBottom), int(letterLeft):int(letterRight)]
    show(letterimage)



    bletterimage = cv2.medianBlur(letterimage,51)

    #basically what happens below is i change all values pretty close to 255 to 255 to prevent artifacts when doing the hist equalize
    bletterimage = cv2.bitwise_not(bletterimage)
    bletterimage = cv2.threshold(bletterimage, 40, 255, cv2.THRESH_TOZERO)[1]
    bletterimage = cv2.bitwise_not(bletterimage)
    # show(bletterimage)


    contrastletterimage = cv2.equalizeHist(bletterimage)
    
    # show(contrastletterimage)
    
    # thresh = np.average(contrastletterimage)*0.5
    thresh = np.average(contrastletterimage)*0.8
    # thresh = 0
    # print(thresh)

    binletterimage = cv2.threshold(bletterimage, int(thresh), 255, cv2.THRESH_BINARY)[1] # + cv2.THRESH_OTSU
    # show(binletterimage)
    
    finalletterimage = cv2.resize(binletterimage, (50, 50))
    show(finalletterimage)








# for folder in "1":
for folder in sorted(os.listdir("2023_Cursive")): #iterate over every folder
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    # for letter in "AB":
        
        image = cv2.imread("2023_Cursive/" + folder + "/" + letter + ".JPG") #bgr to rgb is req for correct colors to show
        show(image)

        #get height and width which end up being used a decent amount for calculating stuff
        height = image.shape[0]
        width = image.shape[1]
        
        gimage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #make img gray
        
        bimage = cv2.medianBlur(gimage,99) #blur using median which gives everything a very smoothed out look which helps a lot with stuff like edge detection
        # show(bimage)
        
        binimage = cv2.adaptiveThreshold(bimage, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 3, 1) #binarize the image,
        # show(binimage)

        edges = cv2.Canny(binimage, 50, 100) #find edges
        # show(edges)
        
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180,200, minLineLength=width*0.5, maxLineGap=height) #find lines from edges
        lineimage = copy.copy(image) #make copy of image specifically for drawing lines and stuff because the original image will be used for cropping and saving later

        #make lists that will hold all vertical and horizontal lines
        vertical = []
        horizontal = []

        try:
            for line in lines:
                for x1,y1,x2,y2 in line: #goes thru every detected line ofc
                    if (x2-x1) != 0: #prevent division by 0 problems
                        slope = (y2-y1)/(x2-x1) 
                    else:
                        slope = (y2-y1)/1 #close enough to a vertical slope
                    
                    if slope > (height/(width*0.2)) or slope < (-height/(width*0.2)): #filters for vertical lines
                        vertical.append((x1,y1,x2,y2)) #adds vertical line to list
                        cv2.line(lineimage,(x1,y1),(x2,y2),(255,0,0),int(width/100)) #vertical lines are blue
                    elif (slope < ((width*0.2)/height) and slope > ((-width*0.2)/height)): #filters for horizontal lines
                        horizontal.append((x1,y1,x2,y2)) #adds horizontal line to list
                        cv2.line(lineimage,(x1,y1),(x2,y2),(0,255,0),int(width/100)) #horizontal lines are blue
        except:
            #save to cropped folder here
            print("no lines detected, assuming already cropped")
            isolateLetter(gimage)
            continue
        
        # show(lineimage)


        
        def line_intersection(line1, line2): #function that gets the intersection of 2 lines
            x1, y1 = line1[0]
            x2, y2 = line1[1]
            x3, y3 = line2[0]
            x4, y4 = line2[1]
            if x2 - x1 != 0:
                slope1 = (y2 - y1) / (x2 - x1)
            else:
                slope1 = (y2 - y1) / 1
            if x3 - x4 != 0:    
                slope2 = (y4 - y3) / (x4 - x3)
            else:
                slope2 = (y4 - y3) / 1
        
            intercept1 = y1 - slope1 * x1
            intercept2 = y3 - slope2 * x3

            if slope1 - slope2 != 0:
                intersection_x = (intercept2 - intercept1) / (slope1 - slope2)
            else:
                intersection_x = (intercept2 - intercept1) / 1
            intersection_y = slope1 * intersection_x + intercept1
        
            return (int(intersection_x), int(intersection_y)) #rounded for simplicity and cv2.circle but doesnt really change anything
            

        sects = [] #list that holds coordinates of every single intersection


        for vertLine in vertical: #only test intersections of vertical lines and horizontal lines, dont test every line against every other line because ofc vertical lines will intercept
            for horizLine in horizontal: 
                vx1, vy1, vx2, vy2 = vertLine
                hx1, hy1, hx2, hy2 = horizLine
                sects.append(line_intersection(((vx1,vy1), (vx2,vy2)), ((hx1,hy1), (hx2,hy2)))) #add all intersections to the sect list


        
        sectimage = copy.copy(image)

        tlSect = []
        trSect = []
        blSect = []
        brSect = []
        
        for sect in sects:
            x = sect[0]
            y = sect[1]
            if x < width/2:
                if y < height/2:
                    tlSect.append(sect)
                else:
                    blSect.append(sect)
            else:
                if y < height/2:
                    trSect.append(sect)
                else:
                    brSect.append(sect)


        
        offset = int(width/20)

        try:

            tlBound = tlSect[0]
            tlBound = (tlBound[0]+offset,tlBound[1]+offset)
            cv2.circle(sectimage, tlBound, int(width*0.001), (0,0,255), int(width*0.03))
    
            blBound = blSect[0]
            blBound = (blBound[0]+offset,blBound[1]-offset)
            cv2.circle(sectimage, blBound, int(width*0.001), (255,255,0), int(width*0.03))
    
            trBound = trSect[0]
            trBound = (trBound[0]-offset,trBound[1]+offset)
            cv2.circle(sectimage, trBound, int(width*0.001), (255,0,0), int(width*0.03))
    
            brBound = brSect[0]
            brBound = (brBound[0]-offset,brBound[1]-offset)
            cv2.circle(sectimage, brBound, int(width*0.001), (0,255,0), int(width*0.03))
        except:
            #save to cropped folder here
            print("missing a corner, assuming already cropped")
            isolateLetter(gimage)
            continue
        
        show(sectimage)


        
        
        corners = np.array([tlBound, trBound, blBound, brBound], dtype=np.float32) #array with 4 coordinate pairs that are the corners of the paper
        
        res = np.array([(0,0), (brBound[0]-tlBound[0]-1,0), (0,brBound[1]-tlBound[1]-1), (brBound[0]-tlBound[0]-1,brBound[1]-tlBound[1]-1)], dtype=np.float32) #will be 200x200 img
        
        ptransform = cv2.getPerspectiveTransform(corners, res) #calculates matrix
        
        warpimage = cv2.warpPerspective(image, ptransform, (brBound[0]-tlBound[0],brBound[1]-tlBound[1])) #applies matrix to img

        show(warpimage)

        gwarpimage = cv2.cvtColor(warpimage, cv2.COLOR_BGR2GRAY) #warped img becomes gray because function needs gray image to be passed in

        isolateLetter(gwarpimage)

        



# Trying a different strategy:

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import copy
import math
import os


for folder in sorted(os.listdir("2023_Cursive")): #iterate over every folder
# for folder in "4": #iterate over every folder
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    # for letter in "AB": #iterate over every file
        
        image = cv2.imread("2023_Cursive/" + folder + "/" + letter + ".JPG") #read img
        gimage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #make img gray
        # blurred = cv2.GaussianBlur(gimage, (9, 9), 0) #blur using gaussian
        # show(blurred)
        blurred = cv2.medianBlur(gimage,65) #blur using median
        show(blurred)

        binimage = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 19, 1)
        show(binimage)

        edges, _ = cv2.findContours(binimage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        c = max(edges, key = cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c)

        cX = x + int(w/2)
        cY = y + int(h/2)
        ROI = gimage[max(cY - 900, 0):min(cY + 900, gimage.shape[1] - 1), max(cX - 700, 0):min(cX + 700, gimage.shape[0] - 1)]
        size = (1400, 1800)
        show(ROI)

        
        imagecrop = cv2.resize(ROI, (ROI.shape[1], ROI.shape[0]), size)
        print("2023_Cursive_Cropped/" + str(folder) + "/" + str(letter) + ".JPG")
        cv2.imwrite("2023_Cursive_Cropped/" + str(folder) + "/" + str(letter) + ".JPG", imagecrop)

In [ ]:
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
import pandas as pd

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score



# flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# 5. Model the data
#### Unfortunately I can't model the data since I never managed to finish the cropping, and had no time.
- I would have tried a neural network.
- Also, I just want to note that I would have to flatten the image to make it fit into a model.
- I would've put everything into a dataframe too.

# 6. Fine Tune the Model

- There isn't really anything to fine tune... because there is no model.

# 7. Present the (theoretical) solution
- First, I imported my data and organized/converted all my files
- Then, I looked through all the pictures I had to see what changes had to be made (cropping, resizing)
- Then, I split the data into a test and train set, and then tried fitting 3 different models to see which would give the best results
- Then, I decided on the model that was best at predicting the data primarily based on RMSE/MAE and maybe some of the plotting data (didn't really use the plotted data since there was no found correlation)
- Finally, deploy the chosen model to a file for usage elsewhere
#### THE CHOSEN MODEL WAS LINEAR REGRESSION ROUNDED TO THE CLOSEST WHOLE INTEGER

# 8. Launch the Model System
- Of course since unfortunately there is no model, there is no way to display the deployment of the ML system.
- What I would do though, is 
